# Image Generation and Display in Browser

## Introduction

This notebook demonstrates generating images from textual prompts using a pre-trained model from the Hugging Face's `diffusers` library. It showcases a simple application of terminal-to-web interfaces, particularly useful for backend developers integrating AI-powered image generation into web applications.

## Setup

First, ensure you have the necessary libraries installed. The primary libraries we'll use are `diffusers` for image generation and `torch` as the backend for the model. If you haven't installed these libraries, uncomment and run the following command.


In [9]:
# Install necessary packages (uncomment if needed)
!pip3 install transformers diffusers torch accelerate

439.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


## Import Libraries

In [10]:
from diffusers import DiffusionPipeline
import torch
import io
import webbrowser
import base64
import tempfile
import os

## Initialize the Diffusion Pipeline

We initialize the diffusion pipeline by loading a pre-trained model to generate images based on textual prompts.


In [11]:
pipeline = DiffusionPipeline.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float32, variant="fp16")
pipeline.to("cpu")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Loading pipeline components...: 100%|██████████| 7/7 [00:16<00:00,  2.38s/it]


StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.25.1",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

## Define Image Processing Functions

We define functions to generate images from prompts and create an HTML page to display these images.


In [12]:
def process_image(prompt):
    """
    Generates an image based on a prompt and returns a Data URI for the image.
    """
    image = pipeline(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
    img_buffer = io.BytesIO()
    image.save(img_buffer, format='PNG')
    img_buffer.seek(0)
    data_uri = base64.b64encode(img_buffer.getvalue()).decode('utf-8')
    return f'data:image/png;base64,{data_uri}'

def create_html_page_with_images(image_urls):
    """
    Creates an HTML page with the given images and opens it in the default web browser.
    """
    # HTML content with basic CSS styling for improved presentation
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Generated Images Gallery</title>
        <style>
            body {
                margin: 0;
                padding: 0;
                background-color: #f0f0f0;
                font-family: Arial, sans-serif;
            }
            .gallery {
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
                padding: 20px;
            }
            .gallery img {
                margin: 10px;
                border: 1px solid #ccc;
                border-radius: 5px;
                width: auto;
                max-width: 300px;
                height: auto;
            }
        </style>
    </head>
    <body>
        <h1 style="text-align: center; margin-top: 20px;">AI Generated Images Gallery</h1>
        <div class="gallery">
    """
    for url in image_urls:
        html_content += f'<img src="{url}" alt="Generated Image"/>'
    html_content += "</div></body></html>"
    with tempfile.NamedTemporaryFile(delete=False, suffix=".html", mode="w", encoding="utf-8") as temp_file:
        temp_file.write(html_content)
        return temp_file.name

### `display_in_browser` Function

The `display_in_browser` function is designed to open a specified HTML file in the default web browser. This is particularly useful for viewing generated content, such as images or reports, directly from a Python script or Jupyter Notebook.

#### Parameters:

- `html_file_path`: A string containing the file path to the HTML file that should be displayed. The path can be absolute or relative to the current working directory.

#### Behavior:

Upon calling, the function constructs a `file://` URL pointing to the specified HTML file path and instructs the web browser to open this file. If successful, the content of the HTML file will be displayed in a new tab or window of the default web browser.

#### Example Usage:

```python
display_in_browser("path/to/your/file.html")


In [13]:
def display_in_browser(html_file_path):
    """
    Opens the given HTML file path in the default web browser.
    """
    webbrowser.open(f'file://{html_file_path}', new=2)

## Running the Demo

This section demonstrates the complete workflow, from generating images based on a given prompt to displaying them in a web browser. Follow the steps below to see the magic happen.

## Prompt for Image Generation
Given the playful theme of our project, we've chosen a whimsical prompt to inspire our AI-powered image generation. The prompt is designed to evoke vibrant and imaginative scenes:

    Prompt: "Create candy land in a handy heaven."

Note: While there is a character limit for the prompt, exceeding it may result in automatic truncation. To ensure a seamless user experience, consider implementing a text completion feature on the backend to refine user inputs, preventing unintended truncation.

## Generating Multiple Images
You have the flexibility to generate up to 10 images per session. However, keep in mind that the rendering time varies depending on system performance. Although running image generation tasks in parallel is feasible, it might lead to resource constraints on less powerful systems. For most users, especially during testing phases, generating 1 to 3 images should provide a satisfactory balance between performance and resource usage.

   number_of_images = 3  # Recommended setting for testing

By tweaking the number_of_images, you can control the output volume to suit your needs, whether for in-depth exploration or quick demonstrations.

NOTE: Before executing below block please seleck the arrow to execute the above code. Sometime the code does register immediately.

In [15]:
if __name__ == "__main__":
    prompt = "Create candy land in a handy heaven."
    number_of_images = 3
    image_urls = [process_image(prompt) for _ in range(number_of_images)]
    html_page_path = create_html_page_with_images(image_urls)
    display_in_browser(html_page_path)

100%|██████████| 1/1 [00:42<00:00, 42.16s/it]


## Conclusion

This notebook demonstrates a simple yet powerful way to integrate backend AI processes with frontend web displays, making it easier for developers to visualize outputs directly in a web browser.
